# Imports

In [1]:
import gc
import pandas as pd
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Felipe\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#data = pd.read_pickle('training_data_new/training_data1.pkl')
#for i in range (2,6):
#    new_data = pd.read_pickle('training_data_new/training_data' + str(i) + '.pkl')
#    data = pd.concat([data, new_data])
    
#data = data.reset_index(drop=True)
#data.to_pickle("training_data_new/training_data_more.pkl.gzip")

# Definitions

In [3]:
DATA_FILE = 'training_data_new/training_data_more.pkl.gzip'

TRAIN_FEATS = [
    'round_num', 'n_pulls_self', 'n_success_self', 'n_failure_self',
       'discounted_cumulative_success', 'discounted_total_success',
       'n_pulls_opp', 'n_pulls_tot', 'ratio_self', 'ratio_opp', 'est_1',
       'est_2', 'est_3', 'repeat_opp', 'n_pulls_self_last_10',
       'n_pulls_opp_last_10', 'ratio_self_last_10_selected'
]

TARGET_COL = 'payout'

# Function to train LGB

In [4]:
gc.collect()

def make_model_lgb(params):
    data = pd.read_pickle(DATA_FILE)
    data['score_agent'] = data['score_agent'] / data['score_agent'].max()
    print(data.shape, '\n')
    
    tr_data = lgb.Dataset(data[TRAIN_FEATS].values, label=data[TARGET_COL], weight = data['score_agent'])
    del data
    gc.collect()
    model = lgb.train(params, tr_data, 92, valid_sets = [tr_data], verbose_eval = 10)
    
    model.save_model(f'model.txt')

# Params

In [5]:
params_lgb = {
                    'objective': 'regression',
                    'boosting_type': 'gbdt',
                    'metric':'rmse',
                    'learning_rate': 0.05,
                    'colsample_bytree': 0.75,
                    'feature_fraction_bynode': 0.75,
                    'max_depth': -1,
                    'num_leaves': 2**10,
                    'subsample': 0.75,
                    'n_jobs':-1,
                    'early_stopping_rounds': 30, 
}

# Run

In [6]:
make_model_lgb(params_lgb)

(231284300, 20) 



C:\Users\Felipe\Anaconda3\lib\site-packages\lightgbm\engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 30 rounds.
[10]	training's rmse: 15.517
[20]	training's rmse: 14.5007
[30]	training's rmse: 14.112
[40]	training's rmse: 13.9648
[50]	training's rmse: 13.9089
[60]	training's rmse: 13.8861
[70]	training's rmse: 13.8753
[80]	training's rmse: 13.8692
[90]	training's rmse: 13.8652
Did not meet early stopping. Best iteration is:
[92]	training's rmse: 13.8647


In [ ]:
#[92]training's rmse: 13.8647 (lgb 1 231 million all features)